# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [21]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/Christian/projects/udacity/data-engineer/Project_Data-Modeling-with-Cassandra
['/Users/Christian/projects/udacity/data-engineer/Project_Data-Modeling-with-Cassandra/event_data/2018-11-15-events.csv', '/Users/Christian/projects/udacity/data-engineer/Project_Data-Modeling-with-Cassandra/event_data/2018-11-22-events.csv', '/Users/Christian/projects/udacity/data-engineer/Project_Data-Modeling-with-Cassandra/event_data/2018-11-09-events.csv', '/Users/Christian/projects/udacity/data-engineer/Project_Data-Modeling-with-Cassandra/event_data/2018-11-18-events.csv', '/Users/Christian/projects/udacity/data-engineer/Project_Data-Modeling-with-Cassandra/event_data/2018-11-04-events.csv', '/Users/Christian/projects/udacity/data-engineer/Project_Data-Modeling-with-Cassandra/event_data/2018-11-01-events.csv', '/Users/Christian/projects/udacity/data-engineer/Project_Data-Modeling-with-Cassandra/event_data/2018-11-27-events.csv', '/Users/Christian/projects/udacity/data-engineer/Project_Data-Model

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


gged In', 'Kinsley', 'F', '33', 'Young', '275.93098', 'paid', 'Red Bluff, CA', 'PUT', 'NextSong', '1.54047E+12', '977', 'Overload', '200', '1.54359E+12', '85'], ['Gyptian', 'Logged In', 'Rylan', 'M', '13', 'George', '233.09016', 'paid', 'Birmingham-Hoover, AL', 'PUT', 'NextSong', '1.54102E+12', '1076', 'Hold You', '200', '1.54359E+12', '16'], ['Zion / Akon', 'Logged In', 'Rylan', 'M', '14', 'George', '233.19465', 'paid', 'Birmingham-Hoover, AL', 'PUT', 'NextSong', '1.54102E+12', '1076', 'The Way She Moves', '200', '1.54359E+12', '16'], ['Transvision Vamp', 'Logged In', 'Rylan', 'M', '15', 'George', '209.91955', 'paid', 'Birmingham-Hoover, AL', 'PUT', 'NextSong', '1.54102E+12', '1076', 'I Want Your Love', '200', '1.54359E+12', '16'], ['Ry Cooder', 'Logged In', 'Chloe', 'F', '37', 'Cuevas', '201.56036', 'paid', 'San Francisco-Oakland-Hayward, CA', 'PUT', 'NextSong', '1.54094E+12', '1096', 'Corrido de Boxeo', '200', '1.54359E+12', '49'], ['The Pussycat Dolls', 'Logged In', 'Rylan', 'M', '

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [35]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [36]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [37]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [18]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [24]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        ## Mapping:
        ## 8: session_id, 3: item_in_session, 0: artist, 9: song, 5: length
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [29]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM music_app_history WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [40]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS music_app_user_session "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [41]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_user_session (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        ## Mapping:
        ## 8: session_id, 3: item_in_session, 0: artist, 9: song, 5: length
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [43]:
query = "SELECT artist, song, first_name, last_name FROM music_app_user_session WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", item_in_session=0, first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', item_in_session=1, first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', item_in_session=2, first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', item_in_session=3, first_name='Sylvie', last_name='Cruz')


In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

### Drop the tables before closing out the sessions

In [30]:
## TO-DO: Drop the table before closing out the sessions
session.execute('DROP TABLE music_app_history')

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()